# [Build an Agent](https://python.langchain.com/v0.2/docs/tutorials/agents/)
==============

前提條件

本指南假設您已熟悉以下概念：

-   [聊天模型](https://python.langchain.com/v0.2/docs/concepts/#chat-models)
-   [工具](https://python.langchain.com/v0.2/docs/concepts/#tools)
-   [代理](https://python.langchain.com/v0.2/docs/concepts/#agents)

單靠語言模型本身無法執行動作——它們只會輸出文本。LangChain 的一個重要應用是創建代理。代理是使用 LLM 作為推理引擎來決定要採取哪些行動以及傳遞哪些輸入的系統。在執行動作後，可以將結果反饋回 LLM 以確定是否需要更多動作，或者是否可以完成。

在本教程中，我們將構建一個可以與搜索引擎互動的代理。您將能夠向這個代理提問，觀察它調用搜索工具，並與它進行對話。

[End-to-end agent](https://python.langchain.com/v0.2/docs/tutorials/agents/#end-to-end-agent "Direct link to End-to-end agent")

In [ ]:
# ! pip3 install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

許多使用 LangChain 開發的應用程序都包含多個步驟，其中涉及多次 LLM 調用。隨著這些應用程序變得越來越複雜，能夠檢查鏈或代理內部發生的情況變得至關重要。而檢查這些過程的最佳方法就是使用 [LangSmith](https://smith.langchain.com/)。

In [ ]:
# export LANGCHAIN_TRACING_V2="true"
# export LANGCHAIN_API_KEY="..."

### [Tavily​](https://python.langchain.com/v0.2/docs/tutorials/agents/#tavily "Direct link to Tavily")

我們將使用 [Tavily](https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/)（一個搜索引擎）作為工具。要使用它，您需要獲取並設置一個 API 密鑰：

In [ ]:
# export TAVILY_API_KEY="..."

### [Define tools​](https://python.langchain.com/v0.2/docs/tutorials/agents/#define-tools "Direct link to Define tools")
--------------------------------------------------------------------------------------------------------------------

我們首先需要創建我們想要使用的工具。我們的主要工具將是 [Tavily](https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/)——一個搜索引擎。在 LangChain 中，我們有一個內建工具，可以輕鬆地將 Tavily 搜索引擎用作工具。

In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("Garmin 手錶")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]
tools

[{'url': 'https://www.garmin.com.tw/products/wearables/?cat=activity_tracking', 'content': 'Garmin 提供多款健康手錶，適合不同的運動需求和生活風格。您可以比較各款手錶的功能，如悠遊卡、音樂播放、AMOLED 螢幕、心率監測等，並找到最適合您的手錶。'}, {'url': 'https://24h.pchome.com.tw/region/DXAO', 'content': 'GARMIN品牌旗艦店，新款手錶Garmin Lily, Venu, Fenix, Vivo智慧手錶、運動手環超值優惠中!還有多款經典熱銷GPS導航、行車記錄器、自行車、路跑、登山、潛水等全系列推薦商品，GARMIN全館商品活動優惠快速到貨'}]


[TavilySearchResults(max_results=2)]

由於 llama3.1 會亂使用工具一通，因此改使用 針對 tools 進行微調的 [llama3-groq-tool-use](https://ollama.com/library/llama3-groq-tool-use) 模型

In [2]:
# import os
# from langchain_openai import AzureChatOpenAI
# model = AzureChatOpenAI(
#     azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
#     azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
#     openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
# )

from langchain_ollama import ChatOllama
model = ChatOllama(model="llama3-groq-tool-use")
model

ChatOllama(model='llama3-groq-tool-use')

In [11]:
from langchain_core.messages import HumanMessage, SystemMessage

我們現在可以看到啟用這個模型進行工具調用的效果。為了實現這一點，我們使用 `.bind_tools` 來讓語言模型了解這些工具。

In [4]:
model_with_tools = model.bind_tools(tools)

我們現在可以調用模型了。首先，我們用一條普通訊息來調用它，看看它如何回應。我們可以查看 `content` 欄位以及 `tool_calls` 欄位。

In [6]:
response = model_with_tools.invoke([HumanMessage(content="Taipei weather?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'Taipei weather'}, 'id': 'e0fc1611-80ac-4c87-83b0-b9704a1f2e71', 'type': 'tool_call'}]


我們可以看到現在沒有文字內容了，但有一個工具調用！它希望我們調用 Tavily 搜索工具。

這還沒有實際調用該工具——它只是告訴我們要這麼做。為了真正調用它，我們需要創建我們的代理。

### [Create the agent​](https://python.langchain.com/v0.2/docs/tutorials/agents/#create-the-agent "Direct link to Create the agent")
--------------------------------------------------------------------------------------------------------------------------------

現在我們已經定義了工具和 LLM，我們可以創建代理。我們將使用 [LangGraph](https://python.langchain.com/v0.2/docs/concepts/#langgraph) 來構建代理。目前，我們正在使用一個高級接口來構建代理，但 LangGraph 的好處在於這個高級接口是由底層的、高度可控的 API 支持的，以防您想修改代理邏輯。

現在，我們可以使用 LLM 和工具來初始化代理。

注意，我們傳入的是 `model`，而不是 `model_with_tools`。這是因為 `create_react_agent` 會在底層為我們調用 `.bind_tools`。

In [7]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

### [Run the agent​](https://python.langchain.com/v0.2/docs/tutorials/agents/#run-the-agent "Direct link to Run the agent")
-----------------------------------------------------------------------------------------------------------------------

我們現在可以在一些查詢上運行代理了！請注意，目前這些都是無狀態查詢（它不會記住之前的交互）。注意，代理在交互結束時會返回最終狀態（其中包含任何輸入，稍後我們將看到如何僅獲取輸出）。

首先，讓我們看看當不需要調用工具時，它如何回應：

In [12]:
response = agent_executor.invoke({"messages": [
    HumanMessage(content="hi!")
]})

response["messages"]

[SystemMessage(content="You have tools with search anything, if you can't answer, use tools.", id='87dfece5-815f-4cdb-b298-9bcfe2552d2b'),
 HumanMessage(content='hi!', id='8b0184fb-a3e9-418c-89fa-441bfa340cd2'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'model': 'llama3-groq-tool-use', 'created_at': '2024-08-13T04:37:23.225689Z', 'message': {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}, 'done_reason': 'stop', 'done': True, 'total_duration': 5269155750, 'load_duration': 31167458, 'prompt_eval_count': 193, 'prompt_eval_duration': 4403705000, 'eval_count': 10, 'eval_duration': 830122000}, id='run-6125aa1f-cfcb-4e7c-b5db-22ce6be5fb6b-0', usage_metadata={'input_tokens': 193, 'output_tokens': 10, 'total_tokens': 203})]

為了確切了解底層發生了什麼（並確保它沒有調用工具），我們可以查看 [LangSmith trace](https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r)。

現在，讓我們嘗試一個應該調用工具的例子。

我們可以查看 [LangSmith trace](https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/r) 來確保它有效地調用了搜索工具。

### [Streaming Messages](https://python.langchain.com/v0.2/docs/tutorials/agents/#streaming-messages "Direct link to Streaming Messages")
--------------------------------------------------------------------------------------------------------------------------------------

我們已經看到如何使用 `.invoke` 調用代理以獲取最終回應。如果代理正在執行多個步驟，這可能需要一段時間。為了顯示中間進度，我們可以在消息發生時將其串流回傳。

In [21]:
messages = [
    SystemMessage(content="You have tools with search anything, perform tools if you can't answer."),
]

In [22]:
for chunk in agent_executor.stream(
    {"messages": messages + [HumanMessage(content="搜尋台北的天氣")]},
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', response_metadata={'model': 'llama3-groq-tool-use', 'created_at': '2024-08-13T04:42:30.775297Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'tavily_search_results_json', 'arguments': {'query': 'Taipei weather'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 7741371542, 'load_duration': 28333584, 'prompt_eval_count': 197, 'prompt_eval_duration': 4629528000, 'eval_count': 32, 'eval_duration': 3067917000}, id='run-37c6b52f-65fe-45df-9a93-03c8f986246c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Taipei weather'}, 'id': '3cd2e00d-d140-4838-8ab8-3f5a4ecc66e1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 197, 'output_tokens': 32, 'total_tokens': 229})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Taipei\', \'region\': \\"T\'ai-pei\\", \'country\': \'Taiwan\', \'lat\': 

### [Streaming tokens](https://python.langchain.com/v0.2/docs/tutorials/agents/#streaming-tokens "Direct link to Streaming tokens")
--------------------------------------------------------------------------------------------------------------------------------

除了串流回傳消息之外，串流回傳 Token 也很有用。我們可以使用 `.astream_events` 方法來實現這一點。

**INFO**

此 `.astream_events` 方法僅適用於 Python 3.11 或更高版本。

In [25]:
async for event in agent_executor.astream_events(
    {"messages": messages + [HumanMessage(content="搜尋台北今天的天氣")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"開始呼叫 Agent: {event['name']} 輸入參數為: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Agent 完成: {event['name']} 輸出參數為: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"開始使用 工具: {event['name']} 輸入參數為: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"工具完成!: {event['name']}")
        print(f"工具輸出參數為: {event['data'].get('output')}")
        print("--")

--
開始使用 工具: tavily_search_results_json 輸入參數為: {'query': 'Taipei weather today'}
工具完成!: tavily_search_results_json
工具輸出參數為: content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Taipei\', \'region\': \\"T\'ai-pei\\", \'country\': \'Taiwan\', \'lat\': 25.04, \'lon\': 121.53, \'tz_id\': \'Asia/Taipei\', \'localtime_epoch\': 1723524175, \'localtime\': \'2024-08-13 12:42\'}, \'current\': {\'last_updated_epoch\': 1723523400, \'last_updated\': \'2024-08-13 12:30\', \'temp_c\': 32.6, \'temp_f\': 90.7, \'is_day\': 1, \'condition\': {\'text\': \'Light rain shower\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/353.png\', \'code\': 1240}, \'wind_mph\': 3.6, \'wind_kph\': 5.8, \'wind_degree\': 57, \'wind_dir\': \'ENE\', \'pressure_mb\': 1006.0, \'pressure_in\': 29.7, \'precip_mm\': 1.54, \'precip_in\': 0.06, \'humidity\': 63, \'cloud\': 55, \'feelslike_c\': 39.5, \'feelslike_f\': 103.1, \'windchill_c\': 32.6, \'windchill_f\': 90.7, \'heatindex_c\': 39.5, \'

### [Adding in memory](https://python.langchain.com/v0.2/docs/tutorials/agents/#adding-in-memory "Direct link to Adding in memory")
--------------------------------------------------------------------------------------------------------------------------------

如前所述，這個代理是無狀態的，這意味著它不會記住之前的交互。為了讓它具有記憶功能，我們需要傳入一個檢查點（checkpointer）。在傳入檢查點時，調用代理時還需要傳入一個 `thread_id`，以便讓代理知道從哪個線程/對話恢復。

In [26]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [27]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [28]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hi Bob! How can I assist you today?', response_metadata={'model': 'llama3-groq-tool-use', 'created_at': '2024-08-13T04:45:24.563512Z', 'message': {'role': 'assistant', 'content': 'Hi Bob! How can I assist you today?'}, 'done_reason': 'stop', 'done': True, 'total_duration': 4787264375, 'load_duration': 39553209, 'prompt_eval_count': 179, 'prompt_eval_duration': 3604494000, 'eval_count': 11, 'eval_duration': 1140247000}, id='run-132d2966-2385-43e2-9883-a5df74f8477c-0', usage_metadata={'input_tokens': 179, 'output_tokens': 11, 'total_tokens': 190})]}}
----


In [29]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Your name is Bob. Is there anything else you'd like to know?", response_metadata={'model': 'llama3-groq-tool-use', 'created_at': '2024-08-13T04:45:31.35337Z', 'message': {'role': 'assistant', 'content': "Your name is Bob. Is there anything else you'd like to know?"}, 'done_reason': 'stop', 'done': True, 'total_duration': 3668375542, 'load_duration': 31225000, 'prompt_eval_count': 204, 'prompt_eval_duration': 2115456000, 'eval_count': 16, 'eval_duration': 1513942000}, id='run-806575f6-54c5-4566-94bf-73e5d5fc4e6f-0', usage_metadata={'input_tokens': 204, 'output_tokens': 16, 'total_tokens': 220})]}}
----


如果我想重新開始對話，只需要變更 `thread_id`

In [30]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I'm sorry, but I can't retrieve information about your personal identity. However, if you have any specific search query in mind, I'd be happy to help you with that!", response_metadata={'model': 'llama3-groq-tool-use', 'created_at': '2024-08-13T04:46:22.103004Z', 'message': {'role': 'assistant', 'content': "I'm sorry, but I can't retrieve information about your personal identity. However, if you have any specific search query in mind, I'd be happy to help you with that!"}, 'done_reason': 'stop', 'done': True, 'total_duration': 8822212458, 'load_duration': 35772041, 'prompt_eval_count': 180, 'prompt_eval_duration': 4558168000, 'eval_count': 38, 'eval_duration': 4224533000}, id='run-9f7e276f-2321-4946-8bcd-245e62fbcf21-0', usage_metadata={'input_tokens': 180, 'output_tokens': 38, 'total_tokens': 218})]}}
----


### [Conclusion​](https://python.langchain.com/v0.2/docs/tutorials/agents/#conclusion "Direct link to Conclusion")
--------------------------------------------------------------------------------------------------------------

結束了！在這個快速入門中，我們介紹了如何創建一個簡單的代理。我們還展示了如何串流回應——不僅包括中間步驟，還包括代幣！我們還加入了記憶功能，這樣您就可以與代理進行對話。代理是一個複雜的主題，有很多東西可以學習！

想了解更多有關代理的信息，請查看 [LangGraph](https://python.langchain.com/v0.2/docs/concepts/#langgraph) 的文檔。這裡有它自己的概念、教程和操作指南。